Import dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import os
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
import re
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [3]:
# src_path = '/content/drive/MyDrive/NMAproject_NLP_toxic/kaggle_dataset'

src_path = '/content/drive/MyDrive/Dataset/train(1).csv'
train_data = pd.read_csv(src_path)
# test_data =pd.read_csv(os.path.join(src_path,'test.csv'))

print(train_data.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


#Preprocess

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', '', text) # Remove IP addresses
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing
train_data['comment_text'] = train_data['comment_text'].apply(preprocess_text)

# Vectorization using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(train_data['comment_text']).toarray()
y_train = train_data.iloc[:, 2:].values

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
sum(X_train)

array([145.02272228, 195.54765771,  10.95653985, ...,  87.97267865,
        43.04873628,  28.89014227])

In [10]:
sum(y_train)

array([15294,  1595,  8449,   478,  7877,  1405])

In [5]:
train_data.head(7)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww match background colour seemingly stuck th...,0,0,0,0,0,0
2,000113f07ec002fd,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,make real suggestion improvement wondered sect...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page,0,0,0,0,0,0
5,00025465d4725e87,congratulation well use tool well talk,0,0,0,0,0,0
6,0002bcb3da6cb337,cocksucker piss around work,1,1,1,0,1,0


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# Train-test split - TODO: remember to remove this cell
X_train, X_val_split, y_train, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [8]:
from sklearn.linear_model import LogisticRegression

In [14]:
# Train Logistic Regression
model = LogisticRegression(max_iter=1000) # multi_class='multinomial', solver='lbfgs
model.fit(X_train, y_train)


# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.linear_model import LogisticRegression

# clf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tfidf, y_train)



# prediction = clf.predict(X_test_tfidf)
# prediction



# from sklearn.metrics import accuracy_score
# print('Accuracy Score: ', accuracy_score(y_test, prediction))



# from sklearn.metrics import hamming_loss
# print('Hamming Loss: ', round(hamming_loss(y_test, prediction),2))

ValueError: y should be a 1d array, got an array of shape (127656, 6) instead.

In [12]:
y_train[0]

array([1, 0, 0, 0, 0, 0])

In [ ]:
y_pred = model.predict(X_val_split)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val_split, y_pred))